In [2]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier, plot_tree
from sklearn.metrics import classification_report
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, classification_report
from sklearn.tree import DecisionTreeClassifier, export_text, plot_tree
from sklearn.linear_model import LogisticRegression
from sklearn.neighbors import KNeighborsClassifier


import matplotlib.pyplot as plt
import seaborn as sns
import acquire
import prepare

import preprocess

Create a new notebook, knn_model, and work with the titanic dataset to answer the following:

Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)

Evaluate your results using the model score, confusion matrix, and classification report.

Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

Run through steps 1-3 setting k to 10

Run through steps 1-3 setting k to 20

What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?

Which model performs best on our out-of-sample data from validate?

In [3]:
train_df, val_df, test_df = prepare.splitting_data(prepare.prep_titanic(acquire.get_titanic_db()), 'survived')

this file exists, reading from csv file


In [4]:
train_encoded, val_encoded, test_encoded = preprocess.preprocess_titanic(train_df, val_df, test_df)

In [5]:
train_encoded.head(3)

,survived,pclass,sibsp,parch,fare,alone,embark_town_Queenstown,embark_town_Southampton,sex_male
776,0,3,0,0,7.750,1,1,0,1
829,1,1,0,0,80.000,1,0,1,0
215,1,1,1,0,113.275,0,0,0,0


In [6]:
X_train = train_encoded.drop(columns='survived')
X_val = val_encoded.drop(columns='survived')
X_test = test_encoded.drop(columns='survived')

In [7]:
y_train = train_encoded.survived
y_val = val_encoded.survived
y_test = test_encoded.survived

In [8]:
X_train.head()

,pclass,sibsp,parch,fare,alone,embark_town_Queenstown,embark_town_Southampton,sex_male
776,3,0,0,7.7500,1,1,0,1
829,1,0,0,80.0000,1,0,1,0
215,1,1,0,113.2750,0,0,0,0
258,1,0,0,512.3292,1,0,0,0
129,3,0,0,6.9750,1,0,1,1


In [9]:
y_train.value_counts()

survived
0    329
1    205
Name: count, dtype: int64

Baseline: survived

If I predicted survived how ofted would be correct?

In [36]:
# Baseline accuracy
y_train.value_counts(normalize=True)[0]

0.6161048689138576

My baseline accuracy is: 61%

In [16]:
knn = KNeighborsClassifier()

In [17]:
knn

KNeighborsClassifier()

In [18]:
knn.fit(X_train, y_train)

KNeighborsClassifier()

Fit a K-Nearest Neighbors classifier to your training sample and transform (i.e. make predictions on the training sample)

Evaluate your results using the model score, confusion matrix, and classification report.

In [19]:
knn.feature_names_in_

array(['pclass', 'sibsp', 'parch', 'fare', 'alone',
       'embark_town_Queenstown', 'embark_town_Southampton', 'sex_male'],
      dtype=object)

In [20]:
knn.classes_

array([0, 1])

In [21]:
y_train.head()

776    0
829    1
215    1
258    1
129    0
Name: survived, dtype: int64

In [23]:
y_pred = knn.predict(X_train)

In [32]:
predicted[:5]

array([0, 1, 1, 1, 0])

In [33]:
knn.predict_proba(X_train)

array([[0.9, 0.1],
       [0.2, 0.8],
       [0.4, 0.6],
       ...,
       [0.4, 0.6],
       [0.4, 0.6],
       [0.6, 0.4]])

In [22]:
knn.score(X_train, y_train)

0.8258426966292135

In [29]:
print(classification_report(y_train, y_pred))

              precision    recall  f1-score   support

           0       0.85      0.87      0.86       329
           1       0.78      0.76      0.77       205

    accuracy                           0.83       534
   macro avg       0.82      0.81      0.81       534
weighted avg       0.82      0.83      0.83       534



positive calculation: 'survived': (1)

In [41]:
count = pd.crosstab(y_train, y_pred)

In [54]:
tp = count.iloc[1,1]
tn = count.iloc[0,0]
fp = count.iloc[0,1]
fn = count.iloc[1,0]

In [51]:
tp, tn, fp, fn

(155, 286, 43, 50)

> Print and clearly label the following: Accuracy, true positive rate, false positive rate, true negative rate, false negative rate, precision, recall, f1-score, and support.

In [57]:
def compute_class_metrics(y_train, y_pred):
    '''
    needed to input y_train, y_pred
    '''
    counts = pd.crosstab(y_train, y_pred)
    TP = counts.iloc[1,1]
    TN = counts.iloc[0,0]
    FP = counts.iloc[0,1]
    FN = counts.iloc[1,0]
    
    all_ = (TP + TN + FP + FN)

    accuracy = (TP + TN) / all_

    TPR = recall = TP / (TP + FN)
    FPR = FP / (FP + TN)

    TNR = TN / (FP + TN)
    FNR = FN / (FN + TP)

    precision =  TP / (TP + FP)
    f1 =  2 * ((precision * recall) / ( precision + recall))

    support_pos = TP + FN
    support_neg = FP + TN
    
    print(f"Accuracy: {accuracy}\n")
    print(f"True Positive Rate/Sensitivity/Recall/Power: {TPR}")
    print(f"False Positive Rate/False Alarm Ratio/Fall-out: {FPR}")
    print(f"True Negative Rate/Specificity/Selectivity: {TNR}")
    print(f"False Negative Rate/Miss Rate: {FNR}\n")
    print(f"Precision/PPV: {precision}")
    print(f"F1 Score: {f1}\n")
    print(f"Support (0): {support_pos}")
    print(f"Support (1): {support_neg}")

In [58]:
compute_class_metrics(y_train, y_pred)

Accuracy: 0.8258426966292135

True Positive Rate/Sensitivity/Recall/Power: 0.7560975609756098
False Positive Rate/False Alarm Ratio/Fall-out: 0.13069908814589665
True Negative Rate/Specificity/Selectivity: 0.8693009118541033
False Negative Rate/Miss Rate: 0.24390243902439024

Precision/PPV: 0.7828282828282829
F1 Score: 0.7692307692307692

Support (0): 205
Support (1): 329


> Run through steps 1-3 setting k to 10

In [66]:
knn10 = KNeighborsClassifier(n_neighbors=10)
knn10.fit(X_train, y_train)
y_pred = knn10.predict(X_train)

In [67]:
compute_class_metrics(y_train, y_pred)

Accuracy: 0.7883895131086143

True Positive Rate/Sensitivity/Recall/Power: 0.6536585365853659
False Positive Rate/False Alarm Ratio/Fall-out: 0.1276595744680851
True Negative Rate/Specificity/Selectivity: 0.8723404255319149
False Negative Rate/Miss Rate: 0.3463414634146341

Precision/PPV: 0.7613636363636364
F1 Score: 0.7034120734908137

Support (0): 205
Support (1): 329


> Run through steps 1-3 setting k to 20

In [70]:
knn20 = KNeighborsClassifier(n_neighbors=20)
knn20.fit(X_train, y_train)
y_pred = knn20.predict(X_train)

In [71]:
compute_class_metrics(y_train, y_pred)

Accuracy: 0.7340823970037453

True Positive Rate/Sensitivity/Recall/Power: 0.5317073170731708
False Positive Rate/False Alarm Ratio/Fall-out: 0.1398176291793313
True Negative Rate/Specificity/Selectivity: 0.8601823708206687
False Negative Rate/Miss Rate: 0.4682926829268293

Precision/PPV: 0.7032258064516129
F1 Score: 0.6055555555555556

Support (0): 205
Support (1): 329


What are the differences in the evaluation metrics? Which performs better on your in-sample data? Why?



> Based on the accuracy, the model with n_neighbors=5 performed the best

Which model performs best on our out-of-sample data from validate?

In [72]:
knn.score(X_train, y_train)

0.8258426966292135

In [73]:
knn10.score(X_train, y_train)

0.7883895131086143

In [74]:
knn20.score(X_train, y_train)

0.7340823970037453

> Based on the validate score, the model with 5 nearest neighbors performed best